## Pydantic Tutorial

[Source](https://docs.pydantic.dev/latest/usage/models/)

* The primary means of defining objects in pydantic is via models (models are simply classes which inherit from BaseModel).

* You can think of models as similar to types in strictly typed languages, or as the requirements of a single endpoint in an API.

* Untrusted data can be passed to a model, and after parsing and validation pydantic guarantees that the fields of the resultant model instance will conform to the field types defined on the model.

### Basic model usage

* `id` is required.
* `name` is not required value but a default value.

In [3]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name = 'Peter Cha'

In [4]:
user = User(id='123') # it works! Auto Data Conversion/Casting.
# cause the validation error
# user_x = User(id = '123.45') 

In [5]:
assert user.id == 123
assert isinstance(user.id, int)
assert user.name == 'Peter Cha'
assert user.__fields_set__ == {'id'}
assert user.dict() == dict(user) == {'id': 123, 'name': 'Peter Cha'}

In [6]:
user.dict()

{'id': 123, 'name': 'Peter Cha'}

In [7]:
# This model is mutable.
user.id = 321
user.id

321

### Model properties

* The example above only shows the tip of the iceberg of what models can do. Models possess the following methods and attributes:


`json()`
* returns a JSON string representation dict(); cf. exporting models

`construct()`
* a class method for creating models without running validation; cf. Creating models without validation

`__fields_set__`
* Set of names of fields which were set when the model instance was initialised

`__fields__`
* a dictionary of the model's fields

`__config__`
* the configuration class for the model, cf. model config

In [8]:
from pprint import pprint

pprint(user.json())
print('----'*10)
pprint(user.construct(id = '123.45'))
print('----'*10)
pprint(user.__fields_set__)
print('----'*10)
pprint(user.__fields__)
print('----'*10)
pprint(user.__config__)
print('----'*10)
pprint(user.schema())

'{"id": 321, "name": "Peter Cha"}'
----------------------------------------
User(id='123.45', name='Peter Cha')
----------------------------------------
{'id'}
----------------------------------------
{'id': ModelField(name='id', type=int, required=True),
 'name': ModelField(name='name', type=str, required=False, default='Peter Cha')}
----------------------------------------
<class '__main__.Config'>
----------------------------------------
{'properties': {'id': {'title': 'Id', 'type': 'integer'},
                'name': {'default': 'Peter Cha',
                         'title': 'Name',
                         'type': 'string'}},
 'required': ['id'],
 'title': 'User',
 'type': 'object'}


### Recursive Models
* More complex hierarchical data structures can be defined using models themselves as types in annotations.



In [9]:
from typing import Optional
from pydantic import BaseModel

In [11]:
class Foo(BaseModel):
    count: int
    size: Optional[float] = None
        
class Bar(BaseModel):
    apple = 'x'
    banana = 'y'
    
class Spam(BaseModel):
    foo: Foo
    bars: list[Bar]

m = Spam(foo={'count': 4}, 
         bars=[{'apple': 'x1'}, {'apple': 'x2'}])

pprint(m)
print('-------'*10)
pprint(m.dict())

Spam(foo=Foo(count=4, size=None), bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')])
----------------------------------------------------------------------
{'bars': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}],
 'foo': {'count': 4, 'size': None}}


### ORM Mode (aka Arbitrary Class Instances)
* Pydantic models can be created from arbitrary class instances to support models that map to ORM(Object Relational Mapping) objects.

To Do this:
1. The Config property `orm_mode` must be set to `True`.
2. The special constructor `from_orm` must be used to create the model instance.

* In the original tutorial uses SQLAlchemy, but I use MongoDB.

* Install MongoDB Community Server via [this guide](https://www.mongodb.com/try/download/community-kubernetes-operator)